# Clase 5 – Enriquecimiento de la capa Plata

En esta notebook se


## Importar las librerías necesarias

In [ ]:
import pandas as pd
from pathlib import Path
import warnings

# Deshabilitar warnings futuros
warnings.simplefilter(action='ignore', category=FutureWarning)

# Ajustar el ancho máximo para impresión en consola
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.width', 300)         # Ajustar a un ancho suficiente en consola
pd.set_option('display.max_colwidth', None) # Evitar recortes en contenido de celdas

print("Importación de librerías completada.")

## Configuración de paths y carpetas del proyecto

In [ ]:
BASE_DIR = Path('..').resolve()
RAW_DIR = BASE_DIR / 'data' / 'raw'
BRONCE_DIR = BASE_DIR / 'data' / 'bronce'
PLATA_DIR = Path("../data/plata")

archivo_plata = PLATA_DIR / "misiones_plata.csv"
archivo_horario = PLATA_DIR / "misiones_horario.csv"

print("Iniciación de carpetas del proyecto completada.")

## Carga del dataset y verificación de estructura

In [ ]:
# Cargar el dataset diario
try:
    df_plata = pd.read_csv(archivo_plata, parse_dates=["FECHA"])
    print("Dataset diario cargado correctamente")
except FileNotFoundError:
    print("El archivo diario no fue encontrado")

# Cargar el dataset horario
try:
    df_horario = pd.read_csv(archivo_horario, parse_dates=["FECHA_HORA"])
    print("Dataset horario cargado correctamente")
except FileNotFoundError:
    print("El archivo horario no fue encontrado")

# Vista preliminar
print("\n Dataset diario:")
df_plata.info()
print("\n")
print(df_plata.head())

print("\n Dataset horario:")
df_horario.info()
print("\n")
print(df_horario.head())

Este paso permite validar la estructura general, tipos de datos y posibles columnas faltantes tanto en el dataset diario como en el horario. Si todo está correcto, avanzaremos con el enriquecimiento.

## Detección y análisis de fechas faltantes

Una vez verificada la estructura del dataset diario, procedemos a identificar si existen fechas faltantes en la serie por estación. 

Esto nos permitirá decidir estrategias para tratar los días sin registros, como imputación o exclusión.


In [ ]:
# Generar el rango completo de fechas esperadas
fechas_totales = pd.date_range(start=df_plata['FECHA'].min(), end=df_plata['FECHA'].max(), freq='D')

# Obtener todas las combinaciones posibles de fecha y estación
estaciones = df_plata['ESTACION'].unique()
index_completo = pd.MultiIndex.from_product([fechas_totales, estaciones], names=['FECHA', 'ESTACION'])

# Reindexar para insertar NaNs explícitos en las fechas faltantes
df_plata = df_plata.set_index(['FECHA', 'ESTACION']).reindex(index_completo).reset_index()

# Verificar fechas faltantes (para exportar listado)
faltantes = df_plata[df_plata.isnull().any(axis=1)][['ESTACION', 'FECHA']]

if not faltantes.empty:
    faltantes.to_csv(PLATA_DIR / "fechas_faltantes.txt", index=False, sep='\t')
    print("Fechas faltantes exportadas a:", PLATA_DIR / "fechas_faltantes.txt")
else:
    print("No se encontraron fechas faltantes")

# Mostrar ejemplo si hay faltantes
print(faltantes.head())

Esta estrategia asegura que cada estación tenga una fila para cada fecha del rango, incluso si originalmente no había registros ese día. Esto deja los valores faltantes como `NaN`, que luego se tratarán.

## Tratamiento de valores nulos

Luego de verificar fechas faltantes, analizamos los valores `NaN` dentro del dataset actual para decidir estrategias de imputación o tratamiento.

### Tratamiento de datos faltantes en el dataset diario

In [ ]:
# Visualizar cantidad de nulos por columna
print("\nValores nulos por columna:")
print(df_plata.isnull().sum())

# Calcular porcentaje de nulos por columna
porcentaje_nulos = df_plata.isnull().mean() * 100
print("\nPorcentaje de valores nulos:")
print(porcentaje_nulos.round(2))

Una vez identificadas las columnas afectadas, proponemos distintas estrategias para completar los datos:

### Relleno con forward fill por estación

In [ ]:
# Ordenar por estación y fecha para aplicar forward fill correctamente
df_plata_ffill = df_plata.sort_values(['ESTACION', 'FECHA']).copy()
df_plata_ffill.update(df_plata.groupby('ESTACION').ffill())

# Vista previa de ejemplo tras forward fill
print("\nEjemplo de datos tras forward fill:")
print(df_plata_ffill.head())

### Imputación con la media de cada estación (solo para columnas numéricas)

In [ ]:
# Imputar con la media por estación
columnas_a_imputar = ['TEMP_MEAN', 'PNM_MEAN', 'HUM_MEAN', 'WIND_SPEED_MEAN', 'WIND_DIR_MEAN']

for col in columnas_a_imputar:
    df_plata_ffill[col] = df_plata_ffill.groupby('ESTACION')[col].transform(lambda x: x.fillna(x.mean()))

# Verificar resultado tras imputación
print("\nValores nulos después de imputación con medias:")
print(df_plata_ffill[columnas_a_imputar].isnull().sum())

Estas estrategias permiten garantizar que las variables derivadas a construir se basen en datos consistentes, sin afectar la distribución ni introducir sesgos evidentes.

### Tratamiento de datos faltantes en el dataset horario

In [ ]:
# Detectar horarios reales de cada estación
df_horario['HORA'] = df_horario['FECHA_HORA'].dt.hour
horarios_por_estacion = df_horario.groupby('NOMBRE')['HORA'].value_counts().unstack(fill_value=0)
horarios_mas_frecuentes = horarios_por_estacion.idxmax(axis=1)

# Detectar horarios outlier (menos del 5% de los días)
outliers_horarios = {}
for estacion in horarios_por_estacion.index:
    total_dias = df_horario[df_horario['NOMBRE'] == estacion]['FECHA_HORA'].dt.date.nunique()
    outliers = horarios_por_estacion.loc[estacion][
        horarios_por_estacion.loc[estacion] / total_dias < 0.05
    ].index.tolist()
    if outliers:
        outliers_horarios[estacion] = outliers

# Crear index completo por estación y sus horarios típicos
df_horario['FECHA'] = df_horario['FECHA_HORA'].dt.floor('D')
estaciones_h = df_horario['NOMBRE'].unique()
fecha_h_min = df_horario['FECHA'].min()
fecha_h_max = df_horario['FECHA'].max()
rango_fechas = pd.date_range(start=fecha_h_min, end=fecha_h_max, freq='D')

# Crear combinaciones válidas por estación
porcentaje_frecuencia = 0.05 # al menos en 5% de los días

index_completo_personalizado = []
for estacion in estaciones_h:
    total_dias_estacion = df_horario[df_horario['NOMBRE'] == estacion]['FECHA'].nunique()
    horas_validas = horarios_por_estacion.columns[
        (horarios_por_estacion.loc[estacion] / total_dias_estacion) >= porcentaje_frecuencia  
    ].tolist()

    for fecha in rango_fechas:
        for hora in horas_validas:
            index_completo_personalizado.append((estacion, pd.Timestamp(fecha + pd.Timedelta(hours=hora))))

index_completo_h = pd.MultiIndex.from_tuples(index_completo_personalizado, names=['NOMBRE', 'FECHA_HORA'])

# Reindexar para insertar valores faltantes en los horarios esperados únicamente
df_horario_completo = df_horario.set_index(['NOMBRE', 'FECHA_HORA']).reindex(index_completo_h).reset_index()

# Verificación
print("\nDiferencia de tamaño (horas originales vs completadas por horario habitual):")
print("Original:", len(df_horario))
print("Completo:", len(df_horario_completo))
print("\nEjemplo de datos horarios con NaN insertados:")
print(df_horario_completo[df_horario_completo.isnull().any(axis=1)].head())



### Mostrar horarios outliers detectados

In [ ]:
# Visualizar registros reales en horarios atípicos detectados
print("\n Registros reales en horarios atípicos:")
for estacion, horas in outliers_horarios.items():
    print(f" - {estacion}: {horas}")

# Registrar los registros reales que ocurren en horarios atípicos
df_outliers_registros = []
for estacion, horas_outlier in outliers_horarios.items():
    registros_outlier = df_horario[
        (df_horario['NOMBRE'] == estacion) &
        (df_horario['HORA'].isin(horas_outlier))
    ]
    if not registros_outlier.empty:
        df_outliers_registros.append(registros_outlier)

# Concatenar y exportar si hay registros
if df_outliers_registros:
    df_outliers_concat = pd.concat(df_outliers_registros)
    archivo_outliers = PLATA_DIR / "registros_horarios_atipicos.csv"
    df_outliers_concat.to_csv(archivo_outliers, index=False)
    print("\n Archivo exportado con registros reales en horarios atípicos:")
    print(archivo_outliers)

## Exportar datasets intermedios (antes de procesar los NaN)

In [ ]:
# Exportar datasets intermedios (si se desea conservar)
df_plata.to_csv(PLATA_DIR / "misiones_plata_con_nan.csv", index=False)
df_plata_ffill.to_csv(PLATA_DIR / "misiones_plata_ffill.csv", index=False)
df_horario_completo.to_csv(PLATA_DIR / "misiones_horario_completo.csv", index=False)

print("Archivos generados correctamente")

In [ ]:
print(df_horario_completo)

## Imputación de datos faltantes basada en promedio entre días anterior y posterior

In [ ]:
# Variables a imputar
variables_objetivo = ['TEMP', 'HUM', 'PNM', 'DD', 'FF']

df_interp = df_horario_completo.copy()

# Asegurar FECHA y HORA correctas
df_interp['FECHA'] = df_interp['FECHA_HORA'].dt.date
df_interp['HORA'] = df_interp['FECHA_HORA'].dt.hour

# Ordenar por estación, fecha y hora
df_interp = df_interp.sort_values(by=['NOMBRE', 'FECHA', 'HORA'])

# Función de imputación por promedio entre día anterior y posterior
def imputar_valores(grupo):
    grupo = grupo.copy()  # para evitar advertencias de SettingWithCopy
    for var in variables_objetivo:
        for idx, fila in grupo.iterrows():
            if pd.isna(fila[var]):
                hora = fila['HORA']
                fecha = fila['FECHA']

                # Buscar el valor del día anterior
                val_ant = grupo[(grupo['HORA'] == hora) & (grupo['FECHA'] < fecha)][var].last_valid_index()
                val_ant = grupo.at[val_ant, var] if val_ant is not None else None

                # Buscar el valor del día posterior
                val_post = grupo[(grupo['HORA'] == hora) & (grupo['FECHA'] > fecha)][var].first_valid_index()
                val_post = grupo.at[val_post, var] if val_post is not None else None

                # Asignar promedio o valor disponible
                if val_ant is not None and val_post is not None:
                    grupo.at[idx, var] = round((val_ant + val_post) / 2, 1)
                elif val_ant is not None:
                    grupo.at[idx, var] = val_ant
                elif val_post is not None:
                    grupo.at[idx, var] = val_post
    return grupo

# Aplicar por estación SIN include_groups
df_interp = (
    df_interp.groupby('NOMBRE', group_keys=False)
    .apply(imputar_valores)
    .reset_index(drop=True)
)

# Redondear valores numéricos a 1 decimal
for var in variables_objetivo:
    df_interp[var] = df_interp[var].round(1)

# Ajustar tipos de columnas
df_interp['HORA'] = df_interp['HORA'].astype('int64')
if 'estacion_archivo' in df_interp.columns:
    df_interp['estacion_archivo'] = df_interp['estacion_archivo'].astype('int64', errors='ignore')

# Exportar
archivo_imputado = PLATA_DIR / "misiones_horario_imputado.csv"
df_interp.to_csv(archivo_imputado, index=False)
print(f"Archivo exportado: {archivo_imputado}")


## Verificar las imputaciones

In [ ]:
# Verificación de imputación final
print("Valores restantes faltantes por variable:")
print(df_interp[variables_objetivo].isnull().sum())

# Vista previa de algunos valores aún faltantes (si existen)
print("\nEjemplos de filas con valores aún faltantes:")
print(df_interp[df_interp[variables_objetivo].isnull().any(axis=1)].head())

In [ ]:
# Contar imputaciones por columna
imputaciones = {}
for var in variables_objetivo:
    # Detectar índices donde original es NaN pero imputado tiene valor
    mask_imputado = df_horario_completo[var].isna() & df_interp[var].notna()
    imputaciones[var] = mask_imputado.sum()

# Mostrar resumen
print("Resumen de imputaciones por variable:")
for var, count in imputaciones.items():
    print(f" - {var}: {count} valores imputados")

# Mostrar ejemplos comparativos (solo filas donde hubo imputación)
print("\nEjemplos de imputaciones realizadas:")
for var in variables_objetivo:
    mask = df_horario_completo[var].isna() & df_interp[var].notna()
    if mask.any():
        print(f"\nVariable: {var}")
        print(df_interp.loc[mask, ['FECHA_HORA', 'NOMBRE', var]])

In [ ]:
# Carpeta con los archivos de días faltantes
FALTANTES_DIR = Path("../data/faltantes")  # Ajustar al directorio correcto
variables_objetivo = ['TEMP', 'HUM', 'PNM', 'DD', 'FF']

# Función para leer días faltantes de un archivo
def leer_dias_faltantes(file_path):
    with open(file_path, "r") as f:
        lines = f.readlines()
    fechas = [line.strip() for line in lines if line.strip() and line.strip()[0].isdigit()]
    return pd.to_datetime(fechas).date

# Recorrer todos los archivos .txt de días faltantes
faltantes_files = list(FALTANTES_DIR.glob("*.txt"))

resumen_resultados = []

for file_path in faltantes_files:
    estacion = file_path.stem.replace("dias_faltantes_", "").replace("_", " ").upper()
    dias_faltantes = leer_dias_faltantes(file_path)
    
    print(f"\n=== Estación: {estacion} ===")
    resultados_estacion = {"Estación": estacion, "Total días faltantes": len(dias_faltantes), "Días completos": 0, "Días con NaN": 0}
    
    for fecha in dias_faltantes:
        subset_original = df_horario_completo[(df_horario_completo['NOMBRE'].str.upper() == estacion) & (df_horario_completo['FECHA'] == fecha)]
        subset_imputado = df_interp[(df_interp['NOMBRE'].str.upper() == estacion) & (df_interp['FECHA'] == fecha)]
        
        if subset_imputado[variables_objetivo].isnull().any().any():
            resultados_estacion["Días con NaN"] += 1
            print(f"\n⚠️ Fecha {fecha} aún con NaN")
        else:
            resultados_estacion["Días completos"] += 1
            print(f"\n✅ Fecha {fecha} imputada correctamente")
        
        # Comparar antes y después
        if not subset_imputado.empty:
            print("\n--- Antes (Original con NaN) ---")
            print(subset_original[['FECHA_HORA', 'NOMBRE'] + variables_objetivo])
            print("\n--- Después (Imputado) ---")
            print(subset_imputado[['FECHA_HORA', 'NOMBRE'] + variables_objetivo])
    
    resumen_resultados.append(resultados_estacion)

# Mostrar resumen final
df_resumen = pd.DataFrame(resumen_resultados)
print("\nResumen de verificación de imputaciones por estación:")
print(df_resumen)

In [ ]:
# Crear un DataFrame reducido solo con días únicos por estación
df_fechas = df_interp[['FECHA', 'NOMBRE']].drop_duplicates()

# Generar rango completo de fechas
fechas_totales = pd.date_range(start=df_fechas['FECHA'].min(), end=df_fechas['FECHA'].max(), freq='D')

# Estaciones
estaciones = df_fechas['NOMBRE'].unique()

# Crear todas las combinaciones posibles (fecha, estación)
index_completo = pd.MultiIndex.from_product([fechas_totales, estaciones], names=['FECHA', 'NOMBRE'])

# Reindexar
df_check = df_fechas.set_index(['FECHA', 'NOMBRE']).reindex(index_completo).reset_index()

# Verificar faltantes
faltantes = df_check[df_check.isnull().any(axis=1)][['NOMBRE', 'FECHA']]

# Exportar resultados
if not faltantes.empty:
    archivo_faltantes_final = PLATA_DIR / "fechas_faltantes_post_imputacion.txt"
    faltantes.to_csv(archivo_faltantes_final, index=False, sep='\t')
    print(f"⚠️ Fechas faltantes exportadas a: {archivo_faltantes_final}")
else:
    print("✅ No se encontraron fechas faltantes después de la imputación.")

# Vista rápida
print("\nEjemplo de fechas faltantes:")
print(faltantes.head())